# Example de simulateur de trading 

### En commance en important les librairies necessaire

In [1]:
import pandas as pd
import datetime
import plotly.express as px

### On lis le csv par python

In [2]:
path = "EURUSD(1).csv"
df = pd.read_csv(path)

### On nettoie le dataframe

In [3]:
df['time'] = pd.to_datetime(df.timestamp, unit='ms')
del(df['timestamp'])
del(df["Unnamed: 0"])
df.set_index("time");

### Creation de la class position

In [4]:
class Position :
    def __init__(self,open_time,open_price,position,takeprofit,lot_size,stop,id=0):
        self.open_time=open_time
        self.open_price=open_price
        self.lot_size=lot_size
        self.position=position
        self.stop=stop
        self.close_price=0
        self.close_time=0
        self.takeprofit=takeprofit
        
        
        if position=='buy':
            self.mul=-1
        if position =='sell':
            self.mul=1
        self.pl=0.0
        self.id=id
        self.calculatedpl=0
        
    def close_position(self,close_time,close_price,ratio):
        self.close_time=close_time
        self.close_price=close_price
        lot_asked=ratio*self.lot_size/100
        self.lot_size=self.lot_size-lot_asked
        self.pl=self.pl+(self.open_price-self.close_price)*self.mul*lot_asked
        self.calculatedpl=1
    def as_dict(self):
        
        return {'open_price':self.open_price,'open_time':self.open_time,'close_time':self.close_time,
                'close_price': self.close_price,'lot_size':self.lot_size,'pl':self.pl,'mul':self.mul}

### On fait la logique pour le test 
#### Et on obtien le resultat

In [6]:
import random

ranger=0.006
open_positions=[]
position_history=[]
pps = []

for i in range(0,len(df)):
    
    if len(pps)<1:
        pps.append(0)
        sss = random.randint(0,1)
        if sss==0 :
            s=random.randint(0,9999999999)
            open_positions.append(Position(df['time'][i],df['close'][i],'buy',df['close'][i]+ranger,1,df['close'][i]-ranger,id=s))
        else :
            s=random.randint(0,9999999999)
            open_positions.append(Position(df['time'][i],df['close'][i],'sell',df['close'][i]-ranger,1,df['close'][i]+ranger,id=s))


    items_to_remove = []
    
    for p in range(len(open_positions)):
        
        if open_positions[p].position=='buy':
            if df['high'][i]>=open_positions[p].takeprofit :
                
                open_positions[p].close_position(df['time'][i],open_positions[p].takeprofit,100)
                position_history.append(open_positions[p])
                items_to_remove.append(open_positions[p])
                
                pps.pop()
                
            elif df['low'][i]<=open_positions[p].stop :
                open_positions[p].close_position(df['time'][i],open_positions[p].stop,100)
                position_history.append(open_positions[p])
                items_to_remove.append(open_positions[p])
                
                pps.pop()
                
                
        elif open_positions[p].position=='sell':
            if df['low'][i]<=open_positions[p].takeprofit :

                open_positions[p].close_position(df['time'][i],open_positions[p].takeprofit,100)
                position_history.append(open_positions[p])
                items_to_remove.append(open_positions[p])
                
                pps.pop()
                
                    
            elif df['high'][i]>=open_positions[p].stop :
                open_positions[p].close_position(df['time'][i],open_positions[p].stop,100)
                position_history.append(open_positions[p])
                items_to_remove.append(open_positions[p])
                
                pps.pop()
                
                            
                        
                        
    open_positions = [position for position in open_positions if position not in items_to_remove]

history=pd.DataFrame([p.as_dict() for p in position_history])
history['total_profit']=history["pl"].cumsum()
chart = px.line(history,x='close_time',y='total_profit')

chart.show()